In [3]:
#load data 
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

x_train_df = pd.read_csv('data_reviews/x_train.csv')
y_train_df = pd.read_csv('data_reviews/y_train.csv')

tr_text_list = x_train_df['text'].values.tolist()
print(len(tr_text_list))


2400


[nltk_data] Downloading package punkt to /home/kali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
"""
Build bag of words 
-tokenization
-set all to lowercase
-remove punctuations
-remove stopwords
"""

#tokenization
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
word_count_vector = cv.fit_transform(tr_text_list)



# print("Number of features: ", len(x.toarray()[0]))
# print(count_vectorizer.get_feature_names())
# print(x.toarray())
# pd.DataFrame(x.toarray(), columns=count_vectorizer.get_feature_names())

appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

# Preprocess Data
def preprocess(review_arr):
    processed = []

    # case characters by converting them all to lowercase
    for text in review_arr:
        processed.append(text.lower())

    # negation handling by converting apostrophes to standard lexicon 
    negation = []
    for text in processed:
        words = text.split()
        reformed = [appos[word] if word in appos else word for word in words]
        reformed = " ".join(reformed)
        negation.append(reformed)

    # tokenize data by converting text to tokens
    tokenized = []
    for text in negation:
        tokenized.append(word_tokenize(text))

    # remove stopwords from sentences (words that are most commonly occurring, but not
    # relevant in the context of the data)
    # we use english stopwords here, which may not be relevant in the context of foreign words
    stop_words = set(stopwords.words('english'))
    for text in tokenized:
        text = [i for i in text if not i in stop_words]

    # remove stand-alone punctuation
    stripped = []
    for text in tokenized:
        words = [word for word in text if word.isalpha()]
        stripped.append(words)
    #return stripped
    # stemming/lemmatization finds the base or dictionary form of the word known as the lemma
    # using vocabulary and morphological analysis
    # use gensim package -- which takes into account part-of-speech as well.
    lemmas = []
    porter = PorterStemmer()
    for text in stripped:
        lemmafied = []
        for t in text:
            lemmafied.append(porter.stem(t))
        lemmas.append(lemmafied)

    return lemmas
  
p = preprocess(tr_text_list)
print(p[14])

['it', 'feel', 'poorli', 'construct', 'the', 'menu', 'are', 'difficult', 'to', 'navig', 'and', 'the', 'button', 'are', 'so', 'recess', 'that', 'it', 'is', 'difficult', 'to', 'push', 'them']


In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer()

# feature transform training set
x_tr_pre = preprocess(tr_text_list)
x_tr = []
for text in x_tr_pre:
    sentence = " ".join(text)
    x_tr.append(sentence)
x_train = vectorizer.fit_transform(x_tr)

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))


# feature transform testing set
x_test_df = pd.read_csv('data_reviews/x_test.csv')
te_text_list = x_test_df['text'].values.tolist()
x_te_pre = preprocess(te_text_list)
x_te = [] 
for text in x_te_pre:
    sentence = " ".join(text)
    x_te.append(sentence)
x_test = vectorizer.transform(x_te)
print(x_train.shape, x_test.shape)
     

/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(2400, 3417) (600, 3417)


In [6]:
#logistic regression
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
import seaborn as sns

logreg = LogisticRegression(solver='liblinear')
# Grid Search for Hyperparameters
penalty = ['l1', 'l2']
C = np.logspace(0, 6, 10)
hyperparams = dict(C=C, penalty=penalty)
clf_LR = GridSearchCV(logreg, hyperparams, cv=5, verbose=0)
clf_LR.fit(x_train, np.ravel(y_train_df))
print("BEST SCORE: ")
print(clf_LR.best_score_)
print("STANDARD DEVIATIONS")
print(clf_LR.cv_results_['std_test_score'])
print("STANDARD DEVIATION FOR BEST SCORE:")
print(clf_LR.cv_results_['std_test_score'][clf_LR.best_index_])


df1= pd.DataFrame(clf_LR.cv_results_)
df1[['rank_test_score','params', 'mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score']].sort_values('rank_test_score')

# pivot = pd.pivot_table(pd.DataFrame(clf.cv_results_), values='mean_test_score', index='param_C', columns='param_penalty')
# ax = sns.heatmap(pivot)

BEST SCORE: 
0.8195833333333333
STANDARD DEVIATIONS
[0.03333333 0.02637207 0.02985497 0.02727815 0.02162657 0.02520747
 0.01475494 0.0247347  0.00857969 0.02496525 0.01065559 0.02202429
 0.01334635 0.02190573 0.01369306 0.01719981 0.016383   0.01481366
 0.01689428 0.01790213]
STANDARD DEVIATION FOR BEST SCORE:
0.027278145505554862


,rank_test_score,params,mean_test_score,std_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
3,1,"{'C': 4.641588833612778, 'penalty': 'l2'}",0.819583,0.027278,0.845833,0.835417,0.785417,0.787500,0.843750
5,2,"{'C': 21.544346900318832, 'penalty': 'l2'}",0.810417,0.025207,0.829167,0.837500,0.775000,0.785417,0.825000
1,3,"{'C': 1.0, 'penalty': 'l2'}",0.809583,0.026372,0.833333,0.808333,0.772917,0.789583,0.843750
2,4,"{'C': 4.641588833612778, 'penalty': 'l1'}",0.801667,0.029855,0.810417,0.827083,0.766667,0.766667,0.837500
7,5,"{'C': 100.0, 'penalty': 'l2'}",0.793333,0.024735,0.814583,0.818750,0.760417,0.766667,0.806250
9,6,"{'C': 464.15888336127773, 'penalty': 'l2'}",0.779167,0.024965,0.806250,0.795833,0.747917,0.750000,0.795833
0,7,"{'C': 1.0, 'penalty': 'l1'}",0.777083,0.033333,0.814583,0.789583,0.722917,0.756250,0.802083
4,8,"{'C': 21.544346900318832, 'penalty': 'l1'}",0.772500,0.021627,0.772917,0.804167,0.747917,0.750000,0.787500
11,9,"{'C': 2154.4346900318824, 'penalty': 'l2'}",0.772500,0.022024,0.800000,0.787500,0.741667,0.752083,0.781250
6,10,"{'C': 100.0, 'penalty': 'l1'}",0.766250,0.014755,0.772917,0.785417,0.750000,0.747917,0.775000


In [7]:
#LR prediction

yproba1_test = clf_LR.predict_proba(x_test)[:, 1]
np.savetxt('yproba1_test.txt', yproba1_test)



In [8]:
# #MLP 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sns

mlp = MLPClassifier(max_iter=120)

hyperparams = {
    'activation': ['identity','logistic', 'relu', 'tanh'],
    'alpha': 10.0 ** -np.arange(-2, 2)
}

clf_mlp = GridSearchCV(mlp, hyperparams, cv=5, verbose=0)
clf_mlp.fit(x_train, y_train_df.values.ravel()) 
print("BEST SCORE: ")
print(clf_mlp.best_score_)
print("STANDARD DEVIATIONS")
print(clf_mlp.cv_results_['std_test_score'])
print("STANDARD DEVIATION FOR BEST SCORE:")
print(clf_mlp.cv_results_['std_test_score'][clf_mlp.best_index_])

df2 = pd.DataFrame(clf_mlp.cv_results_)
df2[['rank_test_score','params', 'mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score']].sort_values('rank_test_score')
# pivot_mlp = pd.pivot_table(pd.DataFrame(clf_mlp.cv_results_), values='mean_test_score', index='param_hidden_layer_sizes', columns='param_activation')

/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (120) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (120) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (120) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (120) reached and the optimization has

BEST SCORE: 
0.81875
STANDARD DEVIATIONS
[0.         0.00083333 0.0247347  0.02867442 0.         0.
 0.05513557 0.02318405 0.         0.0875     0.02674468 0.02504856
 0.         0.01       0.02776389 0.02715056]
STANDARD DEVIATION FOR BEST SCORE:
0.026744677559802022


,rank_test_score,params,mean_test_score,std_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
10,1,"{'activation': 'relu', 'alpha': 1.0}",0.818750,0.026745,0.839583,0.827083,0.789583,0.785417,0.852083
14,1,"{'activation': 'tanh', 'alpha': 1.0}",0.818750,0.027764,0.845833,0.831250,0.787500,0.783333,0.845833
2,3,"{'activation': 'identity', 'alpha': 1.0}",0.818333,0.024735,0.839583,0.831250,0.791667,0.785417,0.843750
7,4,"{'activation': 'logistic', 'alpha': 0.1}",0.809167,0.023184,0.829167,0.804167,0.779167,0.791667,0.841667
11,5,"{'activation': 'relu', 'alpha': 0.1}",0.803750,0.025049,0.833333,0.822917,0.772917,0.775000,0.814583
3,6,"{'activation': 'identity', 'alpha': 0.1}",0.801250,0.028674,0.829167,0.829167,0.758333,0.777083,0.812500
15,7,"{'activation': 'tanh', 'alpha': 0.1}",0.799167,0.027151,0.827083,0.825000,0.758333,0.777083,0.808333
6,8,"{'activation': 'logistic', 'alpha': 1.0}",0.597917,0.055136,0.658333,0.668750,0.556250,0.572917,0.533333
9,9,"{'activation': 'relu', 'alpha': 10.0}",0.543750,0.087500,0.500000,0.500000,0.500000,0.718750,0.500000
13,10,"{'activation': 'tanh', 'alpha': 10.0}",0.505000,0.010000,0.525000,0.500000,0.500000,0.500000,0.500000


In [9]:
#df2[['rank_test_score','params', 'mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score']].sort_values('rank_test_score')

In [10]:

# SVM
from sklearn import tree
#from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import seaborn as sns

#svm = SVC(kernel='linear', C=100)
clf = tree.DecisionTreeClassifier()

# C = np.logspace(0, 6, 10)
# kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
# hyperparams = {
#     'C': C,
#     'kernel': kernels
# }

criterion = ['gini', 'entropy', 'log_loss']
splitter = ['best', 'random']

hyperparams = {
    'criterion': criterion,
    'splitter': splitter
}

clf_mlp = GridSearchCV(clf, hyperparams, cv=5, verbose=0)
clf_mlp.fit(x_train, y_train_df.values.ravel()) 
print("BEST SCORE: ")
print(clf_mlp.best_score_)
print("STANDARD DEVIATIONS")
print(clf_mlp.cv_results_['std_test_score'])
print("STANDARD DEVIATION FOR BEST SCORE:")
print(clf_mlp.cv_results_['std_test_score'][clf_mlp.best_index_])

df3 = pd.DataFrame(clf_mlp.cv_results_)

df3[['rank_test_score','params', 'mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score']].sort_values('rank_test_score')

BEST SCORE: 
0.7270833333333333
STANDARD DEVIATIONS
[0.02624008 0.02575256 0.03468609 0.04501157        nan        nan]
STANDARD DEVIATION FOR BEST SCORE:
0.0450115725860213


/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/home/kali/miniconda3/envs/ml135_env_sp21/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 352, in

,rank_test_score,params,mean_test_score,std_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
3,1,"{'criterion': 'entropy', 'splitter': 'random'}",0.727083,0.045012,0.791667,0.683333,0.672917,0.727083,0.760417
1,2,"{'criterion': 'gini', 'splitter': 'random'}",0.720833,0.025753,0.756250,0.697917,0.704167,0.697917,0.747917
2,3,"{'criterion': 'entropy', 'splitter': 'best'}",0.718750,0.034686,0.760417,0.681250,0.675000,0.747917,0.729167
0,4,"{'criterion': 'gini', 'splitter': 'best'}",0.702917,0.026240,0.743750,0.691667,0.675000,0.681250,0.722917
4,5,"{'criterion': 'log_loss', 'splitter': 'best'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,"{'criterion': 'log_loss', 'splitter': 'random'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df1.to_csv('LR.csv', index=False)
df2.to_csv('nn.csv', index=False)
df3.to_csv('dtree.csv', index=False)

In [15]:

tr_web_list = x_train_df['website_name'].values.tolist()
rst = clf_LR.predict(x_train)

amazon_p = 0
yelp_p = 0
imdb_p = 0
amazon_n = 0
yelp_n = 0
imdb_n = 0

amazon_tp = 0
yelp_tp = 0
imdb_tp = 0
amazon_tn = 0
yelp_tn = 0
imdb_tn = 0

amazon_fp = 0
yelp_fp = 0
imdb_fp = 0
amazon_fn = 0
yelp_fn = 0
imdb_fn = 0

amazon_avg_rev = 0
yelp_avg_rev = 0
imdb_avg_rev = 0

for i in range(0,2400):
    if tr_web_list[i] == "amazon":
        amazon_avg_rev+=x_train[i].shape[0]
    elif tr_web_list[i] == "yelp":
        yelp_avg_rev+=x_train[i].shape[0]
    elif tr_web_list[i] == "imdb":
        imdb_avg_rev+=x_train[i].shape[0]

amazon_avg_rev = amazon_avg_rev/2400
yelp_avg_rev = yelp_avg_rev/2400
imdb_avg_rev = imdb_avg_rev/2400

for i in range(0,2400):
    if y_train_df.values[i] == 1 and tr_web_list[i] == "amazon":
        amazon_p+=1
    elif y_train_df.values[i] == 1 and tr_web_list[i] == "yelp":
        yelp_p+=1
    elif y_train_df.values[i] == 1 and tr_web_list[i] == "imdb":
        imdb_p+=1
    elif y_train_df.values[i] == 0 and tr_web_list[i] == "amazon":
        amazon_n+=1
    elif y_train_df.values[i] == 0 and tr_web_list[i] == "yelp":
        yelp_n+=1
    elif y_train_df.values[i] == 0 and tr_web_list[i] == "imdb":
        imdb_n+=1

for i in range(0,2400):
    if rst[i] == 1 and y_train_df.values[i] == 1 and tr_web_list[i] == "amazon":
        amazon_tp+=1
    elif rst[i] == 1 and y_train_df.values[i] == 1 and tr_web_list[i] == "yelp":
        yelp_tp+=1
    elif rst[i] == 1 and y_train_df.values[i] == 1 and tr_web_list[i] == "imdb":
        imdb_tp+=1
    elif rst[i] == 0 and y_train_df.values[i] == 0 and tr_web_list[i] == "amazon":
        amazon_tn+=1
    elif rst[i] == 0 and y_train_df.values[i] == 0 and tr_web_list[i] == "yelp":
        yelp_tn+=1
    elif rst[i] == 0 and y_train_df.values[i] == 0 and tr_web_list[i] == "imdb":
        imdb_tn+=1
    elif rst[i] == 1 and y_train_df.values[i] == 0 and tr_web_list[i] == "amazon":
        amazon_fn+=1
    elif rst[i] == 1 and y_train_df.values[i] == 0 and tr_web_list[i] == "yelp":
        yelp_fn+=1
    elif rst[i] == 1 and y_train_df.values[i] == 0 and tr_web_list[i] == "imdb":
        imdb_fn+=1
    elif rst[i] == 0 and y_train_df.values[i] == 1 and tr_web_list[i] == "amazon":
        amazon_fp+=1
    elif rst[i] == 0 and y_train_df.values[i] == 1 and tr_web_list[i] == "yelp":
        yelp_fp+=1
    elif rst[i] == 0 and y_train_df.values[i] == 1 and tr_web_list[i] == "imdb":
        imdb_fp+=1

print("OVERALL ACCURACY")
print(".81875")
print("AMAZON ACCURACY")
print(str((amazon_tp+amazon_tn)/(amazon_n+amazon_p)))
print("YELP ACCURACY")
print(str((yelp_tp+yelp_tn)/(yelp_n+yelp_p)))
print("IMDB ACCURACY")
print(str((imdb_tp+imdb_tn)/(imdb_n+imdb_p)))
print("AMAZON TRUE POSITIVE")
print(str((amazon_tp)/(amazon_p)))
print("YELP TRUE POSITIVE")
print(str((yelp_tp)/(yelp_p)))
print("IMDB TRUE POSITIVE")
print(str((imdb_tp)/(imdb_p)))
print("AMAZON TRUE NEGATIVE")
print(str((amazon_tn)/(amazon_n)))
print("YELP TRUE NEGATIVE")
print(str((yelp_tn)/(yelp_n)))
print("IMDB TRUE NEGATIVE")
print(str((imdb_tn)/(imdb_n)))
print("AMAZON FALSE POSITIVE")
print(str((amazon_fp)/(amazon_p)))
print("YELP FALSE POSITIVE")
print(str((yelp_fp)/(yelp_p)))
print("IMDB FALSE POSITIVE")
print(str((imdb_fp)/(imdb_p)))
print("AMAZON FALSE NEGATIVE")
print(str((amazon_fn)/(amazon_n)))
print("YELP FALSE NEGATIVE")
print(str((yelp_fn)/(yelp_n)))
print("IMDB FALSE NEGATIVE")
print(str((imdb_fn)/(imdb_n)))

OVERALL ACCURACY
.81875
AMAZON ACCURACY
0.975
YELP ACCURACY
0.9775
IMDB ACCURACY
0.95875
AMAZON TRUE POSITIVE
0.97
YELP TRUE POSITIVE
0.965
IMDB TRUE POSITIVE
0.9425
AMAZON TRUE NEGATIVE
0.98
YELP TRUE NEGATIVE
0.99
IMDB TRUE NEGATIVE
0.975
AMAZON FALSE POSITIVE
0.03
YELP FALSE POSITIVE
0.035
IMDB FALSE POSITIVE
0.0575
AMAZON FALSE NEGATIVE
0.02
YELP FALSE NEGATIVE
0.01
IMDB FALSE NEGATIVE
0.025
